In [1]:
import json
import requests
import re
from datetime import datetime
import time

In [2]:
BASE_URL = 'https://api.opendota.com/api/'
TEAMS = 'teams'
HEROES = 'heroes'
PROPLAYERS = 'proPlayers'
PROMATCHES = 'proMatches'
MATCHES = 'matches'
PLAYERS = 'players'
HEROES_STATS = 'heroStats'
API_KEY = 

In [3]:
def get_all_teams_from_opendota():
    
    response = requests.get(BASE_URL + TEAMS + '?api_key=' + API_KEY)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

Get all the teams from OpenDota

In [4]:
all_teams_json = get_all_teams_from_opendota()

A sample showing the first value in the list. The items in the list is in a dictionary format and will have keys such as team_id, rating, wins, losses, etc....

In [5]:
all_teams_json[0]

{'team_id': 8291895,
 'rating': 1624.51,
 'wins': 169,
 'losses': 98,
 'last_match_time': 1667130565,
 'name': 'Tundra Esports',
 'tag': 'Tundra',
 'logo_url': 'https://steamusercontent-a.akamaihd.net/ugc/1771573722041415896/D98163DE6281550D35494CEFDF6257F9716BD43B/'}

The function search_item can look for a given string within a list of dictionaries

In [6]:
def search_regex(search_string, name_from_list):
    try:
        x = re.search(search_string, name_from_list, re.IGNORECASE).group()
    except Exception:
        return False
    return x

def search_item(item_to_search, item_key, data_source):
    item_filter =  filter(lambda y: search_regex(item_to_search, y[item_key]), data_source)
    return [item_info for item_info in item_filter]

A sample below searches for the word "Fnatic" from "name" key within the all_teams_json collection. It returned 2 results, manually select the right one

In [11]:
search_item('evil geniuses', 'name', all_teams_json)

[{'team_id': 39,
  'rating': 1413.61,
  'wins': 1257,
  'losses': 767,
  'last_match_time': 1666339052,
  'name': 'Evil Geniuses',
  'tag': 'EG',
  'logo_url': 'https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/39.png'}]

In [8]:
def get_team_stat(team_id):
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '?api_key=' + API_KEY)

    try:
        team_stat_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_stat_json
    
    return 0

In [9]:
def get_match_info(match_id):
    response = requests.get(BASE_URL + MATCHES + '/' + str(match_id) + '?api_key=' + API_KEY)

    try:
        match_info_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
        
    else:
        return match_info_json
    
    return 0

In [10]:
def get_player_info(account_id):
    response = requests.get(BASE_URL + PLAYERS + '/' + str(account_id) + '?api_key=' + API_KEY)

    try:
        player_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return player_data_json
    
    return 0

In [11]:
def get_team_matches(team_id):
    
    response = requests.get(BASE_URL + TEAMS + '/' + str(team_id) + '/' + MATCHES + '?api_key=' + API_KEY)

    try:
        team_data_json = json.loads(response.content.decode('utf-8'))

    except requests.exceptions.Timeout:
        print('request is taking too long to complete, possible timeout, getting old data from Firebase')

    except requests.exceptions.RequestException as e:
        raise SystemExit(e)
    
    else:
        return team_data_json
    
    return 0

Get all the team's matches using their team_id

In [12]:
og_matches_json = get_team_matches(2586976)

This shows the total number of matches for that team

In [13]:
len(og_matches_json)

1360

Here's a sample of a match data 

In [14]:
og_matches_json[0]

{'match_id': 6818704228,
 'radiant_win': True,
 'radiant': False,
 'duration': 1988,
 'start_time': 1666507727,
 'leagueid': 14268,
 'league_name': 'The International 2022',
 'cluster': 154,
 'opposing_team_id': 2163,
 'opposing_team_name': 'Team Liquid',
 'opposing_team_logo': 'https://steamcdn-a.akamaihd.net/apps/dota2/images/team_logos/2163.png'}

Take note that 'start_time' is in Unix (Epoch) time format. It can be converted to standard format by using the 'datetime' library

In [15]:
def oneminutecountdown():
    for i in range(62, -1, -1):
        print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
        time.sleep(1)

def combine_team_matches_data(team_matches_json, predict_team_id):  

    count = 1

    matches_total = len(team_matches_json)

    # 1 x API CALL
    predict_team_stat = get_team_stat(predict_team_id)

    for i in range(60, -1, -1):
        print('Exceeded max API calls, please wait:', i, 'seconds                                                ', end = '\r')
        time.sleep(1)
    
    for index, match_info in enumerate(team_matches_json):
         
        if (count >= 1200):
            oneminutecountdown()
            count = 1
        
        # 1 x API CALL
        match_info_detailed = get_match_info(match_info['match_id'])
        count += 1

        if (count >= 1190):
            oneminutecountdown()
            count = 1
        
        opposing_players = []
        predict_players = []
        
        for player in match_info_detailed['players']:

            opposing_players_dict = {}
            predict_players_dict = {}

            if player['isRadiant'] != match_info['radiant']:
                opposing_players_dict['account_id'] = player['account_id']

                try:
                    opposing_players_dict['name'] = player['name']
                except:
                    opposing_players_dict['name'] = 'NULL'

                opposing_players_dict['player_stats'] = get_player_info(player['account_id'])

                opposing_players.append(opposing_players_dict)
            
            elif player['isRadiant'] == match_info['radiant']:
                predict_players_dict['account_id'] = player['account_id']

                try:
                    predict_players_dict['name'] = player['name']
                except:
                    predict_players_dict['name'] = 'NULL'

                predict_players_dict['player_stats'] = get_player_info(player['account_id'])

                predict_players.append(predict_players_dict)


        count += 10

        team_matches_json[index]['predict_players'] = predict_players
        team_matches_json[index]['opposing_players'] = opposing_players

        if (count >= 1200):
            oneminutecountdown()
            count = 1
        
        # 1 x API CALL
        opposing_team_stat = get_team_stat(match_info['opposing_team_id'])
        count += 1
    
        opposing_team = [{'opposing_team_wins_total' : opposing_team_stat['wins'], 
                        'opposing_team_losses_total' : opposing_team_stat['losses'], 
                        'opposing_team_rating' : opposing_team_stat['rating']}]
        
        predict_team = [{'predict_team_wins_total' : predict_team_stat['wins'], 
                        'predict_team_losses_total' : predict_team_stat['losses'], 
                        'predict_team_rating' : predict_team_stat['rating'],
                        'predict_team_name' : predict_team_stat['name'], 
                        'predict_team_id' : predict_team_stat['team_id']}]
        
        team_matches_json[index]['opposing_team_stat'] = opposing_team
        team_matches_json[index]['predict_team_stat'] = predict_team

        team_matches_json[index]['dire_score'] = match_info_detailed['dire_score']
        team_matches_json[index]['radiant_score'] = match_info_detailed['radiant_score']

        print('match number:', index, 'downloaded out of', matches_total, ':', matches_total - index, 'matches left to go', round((index / matches_total) * 100, 2),'% complete', end = '\r')
        

In [26]:
with open(r'royal_never_give_up_data.json', 'r', encoding="utf-8") as input_file:

    data = input_file.read()
    royal_never_give_up_matches_json = json.loads(data)

In [32]:
combine_team_matches_data(royal_never_give_up_matches_json[752:], 6209804)

In [31]:
len(royal_never_give_up_matches_json[752])

11

In [33]:
for index, item in enumerate(royal_never_give_up_matches_json):
    if len(item) < 17:
        print(index)
        break

In [34]:
# Save the JSON as a file in the working directory
def save_teams_data_to_file(team_name, team_matches_json):
    
    with open(team_name + '_data.json', 'w', encoding='utf-8') as f:
        json.dump(team_matches_json, f, ensure_ascii=False, indent=4)

In [35]:
save_teams_data_to_file('royal_never_give_up', royal_never_give_up_matches_json)